# <span style="color:purple"> The Economist Semantic Analysis (4)

In [27]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
#For NLP
import nltk
from nltk.corpus import stopwords #For stopwords

import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile

# to build tables
import sys
from astropy.table import Table

# to calculate precisions F-measure
import sklearn

%matplotlib inline

In [2]:
lucem_illud.setupStanfordNLP()

Starting downloads, this will take 5-10 minutes
../stanford-NLP/parser already exists, skipping download
../stanford-NLP/ner already exists, skipping download
../stanford-NLP/postagger already exists, skipping download
../stanford-NLP/core already exists, skipping download
[100%]Done setting up the Stanford NLP collection


In [3]:
import lucem_illud.stanford as stanford

/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


## <span style="color:purple"> For information Extrutions

In [15]:
ECONOMIST_Token = pandas.read_csv('ECONOMIST_Token_Neat.csv')

In [16]:
ECONOMIST_Token = ECONOMIST_Token.drop(columns = ['Unnamed: 0'])

In [17]:
ECONOMIST_Token[:3]

,Year,Month,Day,Title,Tag,Article,Token_Word,Remark_Index
0,2006,1,14,Nuclear proliferation Misreading Iran,Leaders,\n\t\t\tSpecial report Iran's nuclear programm...,"['Special', 'report', 'Iran', ""'s"", 'nuclear',...",31
1,2006,1,14,UN reform Fix it or scrap it,Leaders,\n\t\t\tMexico and the United States Shots acr...,"['Mexico', 'and', 'the', 'United', 'States', '...",58
2,2006,1,14,Spain and its regions A Catalan kerfuffle,Leaders,\n\t\t\tSpain and Catalonia Bad echoes from th...,"['Spain', 'and', 'Catalonia', 'Bad', 'echoes',...",45


In [ ]:
ECONOMIST_Token[ECONOMIST_Token.Tag == ]

In [7]:
Topics = ['Leaders', 'Britain', 'Europe', 'United States', 'The Americas', 
              'Middle East and Africa', 'Asia', 'Obituary',
              'Business', 'Finance and economics', 'Science and technology', 'Books and arts']

years = ['2006','2007','2008','2009','2010']

In [8]:
topics = ['Business', 'Finance and economics', 'Science and technology', 'Books and arts']

serial = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [29]:
y = years[1]  # 0,1,2,3,4
topic = topics[0]  #0,1,2,3

In [30]:
E_DF = ECONOMIST_Token[ECONOMIST_Token.Year ==int(y)][ECONOMIST_Token.Tag == topic]
E_DF = E_DF.reset_index()
del E_DF['index']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [31]:
E_DF['tokenized_sentences'] = E_DF['Article'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])


In [ ]:
E_DF['POS_sents'] = E_DF['tokenized_sentences'].apply(lambda x: stanford.postTagger.tag_sents(x))

In [ ]:
# POS counting function
# input: corpus: the text to be analyzed
# input: countTarget: the POS that you want to count;
# input: top: number of top words tagged as countTarget
# output: the list of top number of top words tagged as countTarget
def POS_Counting(corpus, countTarget, top):
    targetCounts = {}
    for entry in corpus:
        for sentence in entry:
            for ent, kind in sentence:
                if kind != countTarget:
                    continue
                elif ent in targetCounts:
                    targetCounts[ent] += 1
                else:
                    targetCounts[ent] = 1
    sortedTargets = sorted(targetCounts.items(), key = lambda x: x[1], reverse = True)
    return sortedTargets[:20]

In [ ]:
# table function POS_Table
def POS_Table(count_list):
    # count the number of nouns 
    word_NN = []
    count_NN = []
    word_VB = []
    count_VB = []
    word_JJ = []
    count_JJ = []
    for i in range(0,20):
        word_NN.append(POS_Counting(count_list['POS_sents'], 'NN', 20)[i][0])
        count_NN.append(POS_Counting(count_list['POS_sents'], 'NN', 20)[i][1])
        word_VB.append(POS_Counting(count_list['POS_sents'], 'VB', 20)[i][0])
        count_VB.append(POS_Counting(count_list['POS_sents'], 'VB', 20)[i][1])
        word_JJ.append(POS_Counting(count_list['POS_sents'], 'JJ', 20)[i][0])
        count_JJ.append(POS_Counting(count_list['POS_sents'], 'JJ', 20)[i][1])
        t = Table([word_NN, count_NN,word_VB, count_VB,word_JJ, count_JJ], 
                  names=('NN', 'NN-count','VB','VB-count', 'JJ','JJ-count'))
    return t

In [ ]:
POS_Table(E_DF)